<H3>CSE587: Data Intensive Computing</H3>
<H4>Lab 1: Part 3</H4>
Author:<br>
Rajat Rajendra Thosar <br>
Teammate:<br>Shreya Chatterjee

In [ ]:
install.packages("ggplot2")
install.packages("maps")
if(!requireNamespace("devtools")) install.packages("devtools")
devtools::install_github("dkahle/ggmap", ref = "tidyup")
install.packages("stringr")
install.packages("rapport")

# install.packages("devtools")
devtools::install_github("wmurphyrd/fiftystater")
install.packages("RColorBrewer")

require(ggplot2)
library(maps)
library("maptools")
library(ggmap)
library(stringr)
library(fiftystater)
library(RColorBrewer)

register_google(key = Sys.getenv("GMapAPI"))

In [ ]:
set.seed(1)
TweetData1 <- read.csv(file.choose(), header = TRUE, sep = ",")
TweetData2 <- read.csv(file.choose(), header = TRUE, sep = ",")
TweetData3 <- read.csv(file.choose(), header = TRUE, sep = ",")
TweetData4 <- read.csv(file.choose(), header = TRUE, sep = ",")
TweetData5 <- read.csv(file.choose(), header = TRUE, sep = ",")
TweetData6 <- read.csv(file.choose(), header = TRUE, sep = ",")
TweetData7 <- read.csv(file.choose(), header = TRUE, sep = ",")

TweetData <- rbind(TweetData1, TweetData2, TweetData3, TweetData4, TweetData5, TweetData6, TweetData7)
#TweetData <- rbind(TweetData1, TweetData2)
#TweetData <- TweetData1
#TweetData <- TweetData7

#str(TweetData1$place_name)
#SynthTweetData<- TweetData1[which( (TweetData1$place_name != "") & (TweetData1$geo_coords== "NA NA") ), ]
# head(SynthTweetData)

tweet <- data.frame(TweetData1$screen_name, TweetData1$location)
cleanedTweets <- tweet[which(as.character(tweet$TweetData1.location) != "" & length(tweet$TweetData1.location)>2), ]
if(length(cleanedTweets[,1])>2499)
{
 finalTweet <- cleanedTweets[sample(nrow(cleanedTweets), 2499),]   
}else
{
 finalTweet <- cleanedTweets
}

latLongData <- geocode(as.character(finalTweet$TweetData1.location))
latLongData<-as.data.frame(latLongData)
write.csv(latLongData,"LatLongData.csv")
#write.csv(latLongData,"LatLongData2KW.csv")

In [ ]:
latLongData<-read.csv(file.choose(),header=T, sep=',')
latLongData<-data.frame(latLongData$lon,latLongData$lat)

addressesDF<-data.frame()
for (llIdx in 1:length(latLongData[,1]))
{
    print(llIdx)
    dataForRevGC <- as.numeric(latLongData[llIdx,])
    addresses <- revgeocode(dataForRevGC, output="address")
    addressesDF<-rbind(addressesDF,as.data.frame(addresses))
}
write.csv(addressesDF,"Addresses.csv")
#write.csv(addressesDF,"Addresses2KW.csv")

In [ ]:
stateList<-state.abb
stateCount <- list()
addressesDF <- read.csv(file.choose())
#regexp<-"/[AL\,AK\,AZ\,AR\,CA\,CO\,CT\,DE\,FL\,GA\,HI\,ID\,IL\,IN\,IA\,KS\,KY\,LA\,ME\,MD\,MA\,MI\,MN\,MS\,MO\,MT\,NE\,NV\,NH\,NJ\,NM\,NY\,NC\,ND\,OH\,OK\,OR\,PA\,RI\,SC\,SD\,TN\,TX\,UT\,VT\,VA\,WA\,WV\,WI\,WY\ \]/"
for (stateIdx in 1:length(stateList))
{
  tmp <- grepl(stateList[stateIdx],addressesDF$addresses)
  stateCount[stateIdx]<-length(tmp[which(tmp==TRUE)])
}

TweetMapData<-cbind(stateList,stateCount)
write.csv(TweetMapData,"TweetCount.csv")
#write.csv(TweetMapData,"TweetCount2KW.csv")
print("Tweet preprocessing completed!")

In [ ]:
install.packages("plotly")
install.packages("httpuv")

library(plotly)
library(httpuv)

# Read TweetCount.csv
TweetMapData <- read.csv(file.choose())
hoverText<- paste(state.name,TweetMapData$stateCount)
LevelLabels <- c("Level 1","Level 2","Level 3","Level 4","Level 5","Level 6","Level 7","Level 8","Level 9","Level 10")

l <- list(color = toRGB("white"), width = 2)
# specify some map projection/options
g <- list(
  scope = 'usa',
  projection = list(type = 'albers usa'),
  showlakes = TRUE,
  lakecolor = toRGB('white')
)

p <- plot_geo(TweetMapData, locationmode = 'USA-states') %>%  
add_trace(
    z = TweetMapData$stateCount, text=hoverText, locations=state.abb,
    color = TweetMapData$stateCount, colors = 'Purples') %>%
    colorbar(title = "Tweet Intensity") %>%
  layout(
    title = '2019 US Influenza Tweets by State<br>(Hover for breakdown)',
    geo = g
  )

p

<h3>Replotting CDC Heatmap for comparisons</h3>

In [ ]:
# install.packages("devtools")
devtools::install_github("wmurphyrd/fiftystater")
#install.packages("RColorBrewer")

library(ggplot2)
library(fiftystater)
library(stringr)
library(RColorBrewer)

data("fifty_states")

FluData<-read.csv(file.choose(), header = TRUE, sep = ",")
SynthFluData <- FluData[which(FluData$WEEK==4),]
SynthFluData$ACTIVITY.LEVEL <- as.character(SynthFluData$ACTIVITY.LEVEL)
LevelLabels <- c("Level 1","Level 2","Level 3","Level 4","Level 5","Level 6","Level 7","Level 8","Level 9","Level 10")

for (obsIdx in 1:length(SynthFluData$ACTIVITY.LEVEL)) {
  tmpString<- as.character(SynthFluData$ACTIVITY.LEVEL[obsIdx])
  SynthFluData$ACTIVITY.LEVEL[obsIdx]<-substr(tmpString,7,str_length(tmpString))
}

SynthFluData$ACTIVITY.LEVEL <- as.numeric(SynthFluData$ACTIVITY.LEVEL)

p2 <- ggplot(SynthFluData, aes(map_id = tolower(SynthFluData$STATENAME))) + 
  # map points to the fifty_states shape data
  geom_map(aes(fill = as.factor(ACTIVITY.LEVEL)), map = fifty_states) + 
  expand_limits(x = fifty_states$long, y = fifty_states$lat) +
  coord_map() +
  scale_fill_brewer(palette = "RdYlGn",labels=LevelLabels, direction = -1) +
  borders(database = "state", regions = ".", fill = NA,
          colour = "black", xlim = NULL, ylim = NULL)+
  scale_x_continuous(breaks = NULL) + 
  scale_y_continuous(breaks = NULL) +
  labs(x = "", y = "",title="2018-19 Influenza Season Week 4 ending Jan 26, 2019") + 
  guides(fill=guide_legend(title="ILI Activity Level"))+
  theme(legend.position = "right",
        panel.background = element_rect(fill='skyblue', colour = 'skyblue'))
p2